In [2]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LrCodeAlong').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema = True, header = True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App',
                                       'Time on Website','Length of Membership'],
                           outputCol='features')

In [13]:
output = assembler.transform(data)

In [15]:
#output.printSchema() #notice newcolumn at the end in added
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [16]:
finalData = output.select('features','Yearly Amount Spent')

In [22]:
finalData.head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]), Yearly Amount Spent=587.9510539684005)]

In [24]:
trainData,testData = finalData.randomSplit([0.7,0.3])

In [25]:
trainData.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean| 498.11658517923723|
| stddev|  78.91655082491867|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
testData.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean| 502.21745188935205|
| stddev|  80.47132437866563|
|    min| 298.76200786180766|
|    max|  725.5848140556806|
+-------+-------------------+



In [28]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [29]:
lrModel = lr.fit(trainData)

In [30]:
result = lrModel.evaluate(testData)

In [32]:
result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.139396177262483|
|-0.4154069268598164|
| -4.098273022390856|
|  6.506449607123841|
|-21.619926718024203|
|  -7.67673773071715|
| -4.167129555102747|
| -9.476076781429072|
|-17.660251040586672|
| -7.471156279827142|
|  7.175680599281975|
|-10.394997817519538|
| -17.30942757040299|
| 11.443820975747599|
|  5.222888230704598|
|  7.973999446554103|
|  5.013109557475218|
| 16.987640570668646|
| 1.2131872296395159|
| 5.1138182286393885|
+-------------------+
only showing top 20 rows



In [34]:
result.rootMeanSquaredError

9.70856468564724

In [35]:
result.r2

0.9853440959578171

In [36]:
finalData.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [37]:
unlabeledData = testData.select('features')

In [38]:
unlabeledData.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.5743636841713...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.4252268808548...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.7207699002873...|
|[31.8209982016720...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9096268275227...|
|[31.9480174211613...|
|[31.9549038566348...|
|[31.9764800614612...|
|[32.0180740106320...|
|[32.0444861274404...|
|[32.0478009788678...|
+--------------------+
only showing top 20 rows



In [39]:
predictions = lrModel.transform(unlabeledData)

In [40]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.0682659804561|
|[30.5743636841713...|442.47982068492547|
|[30.8794843441274...|494.30487300724553|
|[30.9716756438877...| 488.1321601497689|
|[31.1239743499119...|   508.56698055779|
|[31.1280900496166...| 564.9294244777718|
|[31.4252268808548...| 534.9338482098647|
|[31.5261978982398...| 418.5706029737669|
|[31.5702008293202...| 563.6057431819916|
|[31.7207699002873...| 546.2460897578501|
|[31.8209982016720...| 417.4996004139314|
|[31.8648325480987...| 450.2862782943332|
|[31.9048571310136...| 491.2592849932191|
|[31.9096268275227...| 552.0022146974916|
|[31.9480174211613...| 456.6979886621932|
|[31.9549038566348...|432.02388049337287|
|[31.9764800614612...|  325.581336476625|
|[32.0180740106320...|340.79547017464665|
|[32.0444861274404...|447.01664195691046|
|[32.0478009788678...|508.33675295745707|
+--------------------+------------